In [1]:
import ipywidgets as widgets
from ipywidgets import interact, Layout
from src.post_processing import PathWrangler
from IPython.display import display, SVG
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
path_filepath = '../artifacts/processed_expansions/found_paths.json'
predicted_reactions_filepath = "../artifacts/processed_expansions/predicted_reactions.json"
known_reactions_filepath = "../artifacts/processed_expansions/known_reactions.json"

pw = PathWrangler(
    path_filepath=path_filepath,
    pr_filepath=predicted_reactions_filepath,
    kr_filepath=known_reactions_filepath
)

In [3]:
starter_options, target_options = zip(*pw.starter_targets)
target_options = tuple(set(target_options))
starter_selection = list(starter_options)
target_selection = list(target_options)

starter_selector = widgets.SelectMultiple(
    options=starter_options,
    value=starter_options,
    rows=len(starter_options),
    description='Starters:',
    disabled=False
)

def starter_listener(change):
    global starter_selection
    starter_selection = starter_selector.value

starter_selector.observe(starter_listener)

target_selector = widgets.SelectMultiple(
    options=target_options,
    value=target_options,
    rows=len(target_options),
    description='Starters:',
    disabled=False
)

def target_listener(change):
    global target_selection
    target_selection = target_selector.value

target_selector.observe(target_listener)



In [4]:
starter_selector

SelectMultiple(description='Starters:', index=(0, 1, 2, 3, 4, 5, 6), options=('malate', 'pyruvate', 'oxaloacet…

In [5]:
target_selector

SelectMultiple(description='Starters:', index=(0, 1), options=('hopa', 'mvacid'), rows=2, value=('hopa', 'mvac…

In [6]:
evidence_options = {elt.value: elt.name for elt in pw.enzyme_existence}
default_loe = pw.enzyme_existence.PROTEIN.value
loe_selection = [evidence_options[default_loe]]

enzyme_evidence = widgets.SelectMultiple(
    options=evidence_options.keys(),
    value=[default_loe],
    rows=len(evidence_options),
    description='Enzyme level of evidence:',
    disabled=False
)

def evidence_listener(change):
    global loe_selection
    loe_selection = [evidence_options[k] for k in enzyme_evidence.value]

enzyme_evidence.observe(evidence_listener)

enzyme_evidence

SelectMultiple(description='Enzyme level of evidence:', index=(0,), options=('Evidence at protein level', 'Evi…

In [7]:
sort_by_selection = 'mean_rcmcs'
sort_options = ['mean_rcmcs', 'min_rcmcs', 'mdf']
sort_default = 'mean_rcmcs'

sort_by_radio_buttons = widgets.RadioButtons(
    options=sort_options,
    value=sort_default,
    description='Sort paths by:',
    disabled=False
)

def sort_by_listener(change):
    global sort_by_selection
    sort_by_selection = sort_by_radio_buttons.value

sort_by_radio_buttons.observe(sort_by_listener)

sort_by_radio_buttons


RadioButtons(description='Sort paths by:', options=('mean_rcmcs', 'min_rcmcs', 'mdf'), value='mean_rcmcs')

In [8]:
paths = []
def click_handler(b):
    global paths
    filter_by_enzymes = {'existence': loe_selection} # Only paths w/ enzymes w/ protein-level evidence
    paths = pw.get_paths(
        starters=starter_selection,
        targets=target_selection,
        sort_by=sort_by_selection,
        filter_by_enzymes=filter_by_enzymes
    )
    print(f"{len(paths)} paths meet your criteria")

button = widgets.Button(
    description='Get paths',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='solid flask' # (FontAwesome names without the `fa-` prefix)
)

button.on_click(click_handler)

button

Button(button_style='success', description='Get paths', icon='solid flask', style=ButtonStyle(), tooltip='Clic…

10 paths meet your criteria
16 paths meet your criteria


Predicted pathway viewer

In [15]:
current_path = 0
path_slider = widgets.IntSlider(
    min=0,
    max=max(0, len(paths) - 1),
    value=current_path,
    description="Path # "
)

def path_idx_listener(change):
    global current_path
    current_path =  path_slider.value

path_slider.observe(path_idx_listener)

@interact
def display_path(path_idx=path_slider):
    path = paths[path_idx]
    data = ((reaction.image, reaction.top_rcmcs(1)[0]) for reaction in path.reactions)
    print(f"{len(path.reactions)}-step path from: {path.starter} to {path.target}")
    print(f"Min-max driving force = {path.mdf:.2f} kJ/mol\n")
    for i, (img, score) in enumerate(data):
        print(f"----------------------------------------------------")
        print(f"Reaction step #{i+1}")
        print(f"Closest analogue is {score * 100:.2f}% similar")
        display(SVG(img))

interactive(children=(IntSlider(value=0, description='Path # ', max=15), Output()), _dom_classes=('widget-inte…

Known analogues viewer

In [16]:
top_k_slider = widgets.IntSlider(
    min=1,
    max=20,
    value=3,
    description="top-k analogues "
)

@interact
def display_analogues(path_idx=path_slider, k=top_k_slider):
    path = paths[path_idx]
    for i, pr in enumerate(path.reactions):
        svgs = [widgets.HTML(value=SVG(elt.image).data) for elt in pr.top_analogues(k=k)]
        vbox = widgets.VBox(svgs)
        scroll_container = widgets.Box(children=[vbox], layout=widgets.Layout(
                overflow='scroll',
                height='250px',  # Adjust height as needed
                width='2000px'    # Adjust width as needed
            ))
        print(f"----------------------------------------------------")
        print(f"Reaction step #{i+1}")
        display(scroll_container)

interactive(children=(IntSlider(value=0, description='Path # ', max=15), IntSlider(value=3, description='top-k…

Enzyme viewer

In [17]:
'''
path id, reaction step, kth analogue, level of evidence --> enzyme df

'''
current_pr_step = 0
pred_rxn_slider = widgets.IntSlider(
    min=0,
    max=5, # TODO
    value=current_pr_step,
    description="Reaction step: "
)

known_rxn_slider = widgets.IntSlider(
    min=0,
    max=20, #TODO
    value=0,
    description="Analogue #: "
)


@interact
def display_enzymes(path_idx=path_slider, reaction_step=pred_rxn_slider, kth_analogue=known_rxn_slider):
    path = paths[path_idx]
    pr = path.reactions[reaction_step]
    analogues = pr.top_analogues(k=1000)
    es = analogues[kth_analogue].enzymes
    enzyme_df = pd.DataFrame(data=[e.to_dict() for e in es])
    return enzyme_df
    

interactive(children=(IntSlider(value=0, description='Path # ', max=15), IntSlider(value=0, description='React…